# Graph mapping

## Subclassing `GraphModule`

In [1]:
# - Switch off warnings
import warnings

warnings.filterwarnings("ignore")

# - Rockpool imports
import rockpool.graph as rg
from dataclasses import dataclass


@dataclass(eq=False, repr=False)
class MyGraphModule(rg.GraphModule):
    # - Define parameters as for any dataclass
    param1: float
    param2: int
    param3: list

In [2]:
class MyGraphModule(MyGraphModule):
    # - Any initialisation checks can be performed a __post_init__ method
    def __post_init__(self, *args, **kwargs):
        # - You *must* call super().__post_init__()
        super().__post_init__(*args, **kwargs)

        if param1 < param2:
            raise ValueError("param1 must be > param2")

## Transforming `GraphModule` s

In [3]:
import rockpool.graph as rg
from typing import List
from dataclasses import dataclass


@dataclass(eq=False, repr=False)
class MyNeurons(rg.GenericNeurons):
    thresholds: List[int]
    dt: float

    @classmethod
    def _convert_from(cls, mod: rg.GraphModule) -> rg.GraphModule:
        if isinstance(mod, cls):
            # - No need to do anything
            return mod

        elif isinstance(mod, LIFNeuronWithSynsRealValue):
            # - Convert from a real-valued LIF neuron
            # - Get a value for `dt` to use in the conversion
            if mod.dt is None:
                raise ValueError(
                    f"Graph module of type {type(mod).__name__} has no `dt` set, so cannot convert time constants when converting to {cls.__name__}."
                )

            # - Get thresholds from source module
            thresholds = np.round(np.array(mod.threshold)).astype(int).tolist()

            # - Build a new self module to insert into the graph
            neurons = cls._factory(
                len(mod.input_nodes),
                len(mod.output_nodes),
                mod.name,
                thresholds,
                mod.dt,
            )

            # - Replace the target module and return
            rg.replace_module(mod, neurons)
            return neurons

        else:
            raise ValueError(
                f"Graph module of type {type(mod).__name__} cannot be converted to a {cls.__name__}"
            )

## Creating a mapper

## DRC checks

In [4]:
import rockpool.graph as rg
from typing import List, Callable

# - Define an error class for DRC violations
class DRCError(ValueError):
    pass


def output_nodes_have_neurons_as_source(graph: rg.GraphModule):
    # - All output nodes must have a source that is a neuron
    for n in graph.output_nodes:
        for s in n.source_modules:
            if not isinstance(s, rg.GenericNeurons):
                raise DRCError(
                    f"All network outputs must be directly from neurons.\nA network output node {n} has a source {s} which is not a neuron."
                )


def first_module_is_a_weight(graph: rg.GraphModule):
    # - The first module after the input must be a set of weights
    for inp in graph.input_nodes:
        for sink in inp.sink_modules:
            if not isinstance(sink, rg.LinearWeights):
                raise DRCError(
                    f"The network input must go first through a weight.\nA network input node {inp} has a sink module {sink} which is not a LinearWeight."
                )


def le_16_input_channels(graph: rg.GraphModule):
    # - Only 16 input channels are supported
    if len(graph.input_nodes) > 16:
        raise DRCError(
            f"Xylo only supports up to 16 input channels. The network requires {len(graph.input_nodes)} input channels."
        )

In [5]:
# - Collect a list of DRC functions
xylo_drc = [
    output_nodes_have_neurons_as_source,
    first_module_is_a_weight,
    le_16_input_channels,
]


def check_drc(graph, design_rules: List[Callable[[rg.GraphModule], None]]):
    """
    Perform a design rule check
    """
    for dr in design_rules:
        try:
            dr(graph)
        except DRCError as e:
            raise DRCError(
                f"Design rule {dr.__name__} triggered an error:\n"
                + "".join([f"{msg}" for msg in e.args])
            )


# - To perform the DRC check, use the function like so:
# check_drc(graph, xylo_drc)